<a href="https://colab.research.google.com/github/amalvarezme/AprendizajeMaquina/blob/main/8_NLP_Basics/NLP4_Transformers_Generative_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI for Stylistic Text Generation with Transformers

This notebook provides an update on using generative AI, specifically Transformer models, for stylistic text generation. The core idea is to take a piece of text and rewrite it in a different style (e.g., formal, informal, Shakespearean) while preserving the original meaning.

We will explore and compare three different approaches using powerful pre-trained models, leveraging their learned embeddings and architectures. All examples will use **TensorFlow** and **Keras**.

### The Three Models We'll Compare:

1.  **GPT-2 ([Generative Pre-trained Transformer 2](https://openai.com/index/gpt-2-1-5b-release/)):** A decoder-only, autoregressive model. We'll use it for stylistic generation via **prompt engineering**.
2.  **[BERT-based Encoder-Decoder](https://arxiv.org/pdf/1810.04805) (Bidirectional Encoder Representations from Transformers):** A hybrid approach where we use a pre-trained **BERT** as a fixed encoder (for its powerful contextual embeddings) and build a custom Transformer decoder in Keras to generate the stylistic text.
3.  **T5 ([Text-to-Text Transfer Transformer](https://arxiv.org/pdf/1910.10683)):** An encoder-decoder model that frames every NLP task as a text-to-text problem, making it a natural fit for stylistic transfer.

In [14]:
!pip install -q transformers tensorflow

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Hugging Face Transformers imports
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from transformers import TFBertModel, BertTokenizer
from transformers import TFT5ForConditionalGeneration, T5Tokenizer

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.18.0


## Model 1: GPT-2 for Stylistic Generation via Prompting

GPT-2 is an autoregressive model, meaning it generates text one token at a time based on the preceding tokens. It excels at continuing a piece of text provided as a "prompt". We can leverage this for stylistic generation by carefully crafting the prompt to guide the model's output.

### Architecture & Embedding Approach
*   **Architecture**: Decoder-only Transformer. It consists of a stack of Transformer decoder blocks without the cross-attention mechanism found in encoder-decoder models.
*   **Embedding Approach**: It uses its own pre-trained **Byte-Pair Encoding (BPE)** tokenizer and embedding layer. The model learns rich, contextual embeddings from scratch during its massive pre-training phase.

### Generation Procedure
1.  **Craft a Prompt**: Create a prompt that includes the input text and a clear instruction for the desired style. This is often called "in-context learning" or prompt engineering.
2.  **Tokenize**: Convert the prompt into token IDs using the GPT-2 tokenizer.
3.  **Generate**: Feed the token IDs to the model and let it generate the continuation autoregressively. We can control the output using parameters like `max_length`, `temperature`, `top_k`, and `no_repeat_ngram_size`.

In [16]:
# Load pre-trained GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Ensure there's a padding token (GPT-2 doesn't have one by default)
if gpt2_tokenizer.pad_token is None:
    gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [17]:
def generate_with_gpt2_style(input_text, style_description):
    """Generates text in a specific style using GPT-2 and prompt engineering."""

    # Craft a clear prompt for the model
    prompt = f"{input_text}\n--- In the style of {style_description}, this would be rewritten as: ---\n"

    print(f"\n--- GPT-2 Prompt ---\n{prompt}\n--------------------\n")

    # Tokenize the prompt
    input_ids = gpt2_tokenizer.encode(prompt, return_tensors='tf')

    # Generate text
    # no_repeat_ngram_size helps prevent repetitive phrases
    # temperature controls randomness: lower is more deterministic
    output_sequences = gpt2_model.generate(
        input_ids=input_ids,
        max_length=len(input_ids[0]) + 50, # Generate 50 new tokens
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )

    # Decode and print the result
    generated_text = gpt2_tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Extract only the newly generated part
    stylistic_output = generated_text.split("--- In the style of")[-1]
    stylistic_output = stylistic_output.split("this would be rewritten as: ---")[-1].strip()

    print(f"GPT-2 Stylistic Output:\n{stylistic_output}")

# --- Example Usage ---
modern_text = "I'm heading to the market to buy some food because I'm very hungry."
generate_with_gpt2_style(modern_text, "a pirate from the 17th century")
generate_with_gpt2_style(modern_text, "a formal scientific paper")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- GPT-2 Prompt ---
I'm heading to the market to buy some food because I'm very hungry.
--- In the style of a pirate from the 17th century, this would be rewritten as: ---

--------------------



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


GPT-2 Stylistic Output:
I've got a whole bunch of things I need to get my hands on. I can't get this thing. My family needs a home, but I don't have enough money to afford it. So I want to find somewhere to live. And

--- GPT-2 Prompt ---
I'm heading to the market to buy some food because I'm very hungry.
--- In the style of a formal scientific paper, this would be rewritten as: ---

--------------------

GPT-2 Stylistic Output:
"The current food supply and demand is determined by the supply of food and the demand for food, not by how much food is available. The supply is the quantity of energy available to produce a given quantity. This is how the present system is calculated


### GPT-2 Analysis

*   **Pros**: Extremely easy to set up and use. No fine-tuning required for decent results, especially with larger models like GPT-3/4. Highly flexible.
*   **Cons**: The model might not strictly follow the stylistic instructions and can hallucinate or go off-topic. The output quality is highly dependent on the prompt's quality. It's not a structured, repeatable process like a sequence-to-sequence model.

## Model 2: BERT-based Encoder-Decoder

This approach is more structured. We use a pre-trained BERT model as a sentence encoder. Its job is to read the input text and convert it into a sequence of rich, contextual embeddings. We then train a separate Transformer decoder from scratch to read these embeddings and generate the text in the target style.

### Architecture & Embedding Approach
*   **Architecture**: A hybrid Encoder-Decoder model.
    *   **Encoder**: A pre-trained BERT model (`TFBertModel`). We freeze its weights to use it solely as a high-quality embedding extractor.
    *   **Decoder**: A custom Keras model, typically built with `layers.TransformerDecoder` or `layers.LSTM`. This part is trained on our specific stylistic task.
*   **Embedding Approach**: BERT provides **contextual word embeddings**. Unlike static embeddings (like Word2Vec), the embedding for the word "bank" is different in "river bank" vs. "money bank". This provides a much richer representation of the source text.

### Generation Procedure
This method requires **training a new model** on a paired dataset (e.g., `(informal_sentence, formal_sentence)`).

1.  **Data Preparation**: Create a dataset of `(source_style, target_style)` pairs.
2.  **Tokenization**: Use a BERT tokenizer for the source text and a separate tokenizer for the target text.
3.  **Model Building**: Construct the Keras model with the frozen BERT encoder and a trainable decoder.
4.  **Training**: Train the decoder to predict the next token in the target sentence, conditioned on the BERT encoder's output and the previously generated tokens.
5.  **Inference**: To generate, encode the source text with BERT, then start with a `[START]` token and autoregressively feed the decoder's output back as its next input until an `[END]` token is produced.

 **Note:** a full implementation requires a proper dataset and training loop, which is beyond the scope of this notebook.

### BERT-based Model Analysis

*   **Pros**: Highly structured and reliable once trained. By using BERT, we leverage a powerful understanding of the source language. Gives you full control over the model architecture and training process.
*   **Cons**: Requires a significant amount of paired data for the target style. Much more complex to implement and train compared to the other two methods. The decoder is trained from scratch, so it needs a good amount of data to become effective.

## Model 3: T5 for Text-to-Text Stylistic Transfer

T5 (Text-to-Text Transfer Transformer) is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks. Its key innovation is framing every NLP problem as a text-to-text task.

### Architecture & Embedding Approach
*   **Architecture**: A standard Transformer Encoder-Decoder model.
*   **Embedding Approach**: Uses [**SentencePiece**](https://arxiv.org/pdf/1808.06226) tokenization, which is effective for handling a wide variety of text and languages. Like GPT-2 and BERT, its embeddings are learned contextually during pre-training.

### Generation Procedure
The procedure is simple and elegant. We prepend a **task-specific prefix** to the input text. To perform stylistic transfer, we just need to define a prefix for our task.

1.  **Define a Prefix**: Create a prefix like `"make this formal: "` or `"rewrite in a Shakespearean style: "`.
2.  **Combine**: Concatenate the prefix with the input text.
3.  **Tokenize and Generate**: Feed the combined text to the T5 model and generate the output.

**Important Note**: For T5 to be truly effective at a custom stylistic task, it must be **fine-tuned** on a dataset where every input is prepended with this prefix. However, we can demonstrate the *mechanism* with the base pre-trained model. Its vast pre-training allows it to sometimes understand these zero-shot instructions surprisingly well.

In [18]:
# Load pre-trained T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [19]:
def generate_with_t5_style(input_text, style_prefix):
    """Generates text in a specific style using T5 and a task prefix."""

    # T5's text-to-text format
    prompt = f"{style_prefix}: {input_text}"

    print(f"\n--- T5 Input ---\n{prompt}\n------------------\n")

    # Tokenize the prompt
    input_ids = t5_tokenizer.encode(prompt, return_tensors='tf')

    # Generate text
    output_sequences = t5_model.generate(input_ids=input_ids, max_length=100)

    # Decode and print the result
    generated_text = t5_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    print(f"T5 Stylistic Output:\n{generated_text}")

# --- Example Usage ---
# Note: The base T5 model wasn't fine-tuned for these specific tasks,
# so its output might be more of a translation or summary.
# However, this demonstrates the powerful text-to-text mechanism.

modern_text = "I'm heading to the market to buy some food because I'm very hungry."

# A task T5 is good at: Translation
generate_with_t5_style(modern_text, "translate English to German")

# Our custom stylistic task (zero-shot attempt)
generate_with_t5_style(modern_text, "make this sound formal")


--- T5 Input ---
translate English to German: I'm heading to the market to buy some food because I'm very hungry.
------------------

T5 Stylistic Output:
Ich reise auf den Markt, um einige Lebensmittel zu kaufen, weil ich sehr hungrig bin.

--- T5 Input ---
make this sound formal: I'm heading to the market to buy some food because I'm very hungry.
------------------

T5 Stylistic Output:
sound formal: I'm heading to the market to buy some food because I'm very hungry.


### T5 Analysis

*   **Pros**: The text-to-text framework is extremely powerful and intuitive for sequence-to-sequence tasks. Fine-tuning is straightforward and effective. A single fine-tuned T5 model can handle multiple tasks by using different prefixes.
*   **Cons**: Requires fine-tuning on a labeled dataset for new tasks to be reliable. The base model might not understand novel stylistic prefixes without this fine-tuning.

## 5. Comparison and Conclusion

Let's summarize the three approaches in a table.

| Feature                  | GPT-2 (Prompting)                                       | BERT + Custom Decoder                                   | T5 (Text-to-Text)                                    |
|--------------------------|---------------------------------------------------------|---------------------------------------------------------|------------------------------------------------------|
| **Core Architecture**    | Decoder-Only                                            | Encoder-Decoder (Hybrid)                                | Encoder-Decoder                                      |
| **Embedding Approach**   | Uses its own pre-trained BPE embeddings.                | Uses pre-trained contextual BERT embeddings (frozen).   | Uses its own pre-trained SentencePiece embeddings.   |
| **Generation Procedure** | Autoregressive continuation of a carefully crafted prompt. | Train a new decoder on paired data to generate output.  | Encodes prefixed input and decodes to generate output. |
| **Setup Complexity**     | **Low**: Load model and start prompting.                  | **High**: Requires custom Keras model and training loop. | **Medium**: Load model, then fine-tune on prefixed data. |
| **Data Requirement**     | **None** (for zero-shot), but prompt quality is key.      | **High**: Needs a large, high-quality paired dataset.   | **Medium**: Needs a paired dataset formatted with prefixes. |
| **Best For**             | Quick experiments, creative generation, few-shot tasks. | Building highly customized, high-performance models.    | Structured Seq2Seq tasks like translation & style transfer. |

### Final Thoughts

Choosing the right model for stylistic generation depends heavily on your project's requirements, available data, and desired level of control.

*   For **rapid prototyping or creative applications** where consistency is not paramount, **GPT-2's prompt engineering** is an excellent starting point.
*   For **building a robust, high-performance system** where you have a large paired dataset and need deep customization, the **BERT + Custom Decoder** approach offers the most control, leveraging the best of pre-trained encoders and custom-trained decoders.
*   For a **balanced, powerful, and scalable solution**, **T5** is often the best choice. Its text-to-text framework is a natural fit for stylistic transfer, and fine-tuning it is a well-established and highly effective process.